In [1]:
# %load_ext autoreload
# %autoreload 2
import os
import torch
from torch import nn
from ASTEncoder import ASTEncoder
from CombinedRepresentation import CombinedRepresentation
from Decoder import Decoder
from Attention import Attention
import torch.nn.functional as F

ModuleNotFoundError: No module named 'ASTEncoder'

In [2]:
import numpy as np
import random
def randContexts(batch_size, max_contexts, context_len):
    return torch.tensor(
        np.random.randint(0, 10, size=(batch_size, max_contexts, context_len))
    )

def randPaddedContexts(batch_size, max_contexts, context_len):
    contexts = []
    lengths = []
    for i in range(0, batch_size * max_contexts):
        rand_len = random.randint(1, context_len)
        
        lengths.append(rand_len)
        context = np.random.randint(1, 10, size=(rand_len))
        
        padded_context = np.zeros(context_len)
        padded_context[:context.shape[0]] = context
        contexts.append(
            padded_context
        )
#     torch.tensor(lengths).reshape(batch_size, max_contexts)
    final_contexts = np.stack(contexts)
    return torch.tensor(
        final_contexts, dtype=torch.long
    ).reshape(batch_size, max_contexts, context_len), lengths
    




In [14]:
max_contexts = 21
batch_size = 5
token_len = 6
token_embedding_size = 7
token_num_embeddings = 20
ast_path_len = 10
ast_num_embeddings = 30
ast_embedding_size = 8
ast_hidden_size = 30
encoder_hidden = 27
ast_bidirectional = True
decoder_hidden = 27
decoder_out = 31
decoder_embedding_size = 7

padded_encoder = CombinedRepresentation(
     max_contexts,
     token_len,
     token_embedding_size,
     token_num_embeddings,
     ast_path_len,
     ast_num_embeddings,
     ast_embedding_size,
     ast_hidden_size,
     encoder_hidden,
     ast_bidirectional=True,
     ast_pad_idx=0
)

attention = Attention(
    encoder_hidden,
    decoder_hidden
)

decoder = Decoder(
    decoder_out,
    decoder_embedding_size,
    encoder_hidden,
    decoder_hidden,
    0.5,
    attention
)

In [17]:
path_contexts, path_lens = randPaddedContexts(batch_size, max_contexts, ast_path_len)
start_contexts, _ = randPaddedContexts(batch_size, max_contexts, token_len)
end_contexts, _ = randPaddedContexts(batch_size, max_contexts, token_len)
decoder_previous = torch.tensor([2] * batch_size)

In [18]:

# embedding = nn.Embedding(ast_num_embeddings, ast_embedding_size, padding_idx=0)
# padded_contexts
# embedded = embedding(padded_contexts)
# embedded = embedded.reshape(batch_size * max_contexts, ast_path_len, ast_embedding_size)

# embedded_packed = torch.nn.utils.rnn.pack_padded_sequence(
#                 embedded,
#                 lengths,
#                 batch_first=True,
#                 enforce_sorted=False,
#             )

In [27]:
def run(encoder, decoder, start, end, path, path_lens, decoder_h_prev):
    encoded = encoder(start, end, path, path_lens)
    decoder_initial = encoded.sum(dim=1) / max_contexts
    decoder_initial.shape
    pred, hidden = decoder(decoder_h_prev, decoder_initial, encoded)
    
    return pred, hidden

# embedded_packed

In [29]:
pred, _ = run(encoder, decoder, start_contexts, end_contexts, path_contexts, path_lens, decoder_previous)
pred_packed, _ = run(padded_encoder, decoder, start_contexts, end_contexts, path_contexts, path_lens, decoder_previous)

not padding


In [7]:
# input = [batch size] the previous prediction or ground truth if using teacher forcing
# hidden = [batch size, dec hid dim]
# encoder_outputs = [src len, batch size, enc hid dim * 2]


In [30]:
pred_packed

tensor([[ 0.0192,  0.2672,  0.3908,  0.1093,  0.2075, -0.4593, -0.1799, -0.9357,
         -0.7444, -0.0676,  0.1363,  0.3206, -0.8241, -0.1877, -0.2045, -0.4505,
         -0.2019, -0.6395,  0.5510, -0.0419, -0.6607,  0.0188,  0.3983,  0.6107,
          0.3753,  0.5021, -0.4667, -0.4846,  0.7062, -0.0253, -0.1780],
        [ 0.0953, -0.0882,  0.3342, -0.3160,  0.5703, -0.1083, -0.1204, -0.7152,
         -0.2773, -0.2063,  0.1920, -0.0682, -0.4073, -0.3183,  0.1385, -0.1170,
          0.2447, -0.0655,  0.2473, -0.0940, -0.3435, -0.0669, -0.0949,  0.5371,
          0.3143,  0.2550, -0.0815, -0.3280,  0.3665, -0.3363,  0.1134],
        [ 0.2581,  0.1374,  0.5975,  0.5561,  0.1268, -1.1876,  0.3828, -0.4699,
         -0.3717,  0.2166,  0.6081,  0.0838, -0.3799, -0.6094, -0.5966, -0.6883,
          0.1776, -0.9161, -0.2207,  0.4199, -0.2658, -0.1434,  0.1715,  1.1473,
         -0.0260,  0.7278,  0.4953, -0.7427,  0.5782,  0.2400, -0.4009],
        [ 0.3627, -0.0037,  0.4645,  0.6881,  0.2761

In [31]:
pred

tensor([[ 0.5193,  0.2451,  0.7190,  0.7391, -0.3771, -0.6199,  0.4121,  0.4511,
         -0.1682,  0.5793,  0.2046,  0.0249, -0.4175, -0.3419, -0.7307, -0.5142,
         -0.4812, -0.9429,  0.1567,  0.3886, -0.0259, -0.2589, -0.1656,  0.6226,
          0.0855,  0.2261,  0.2862, -0.2775,  0.2412,  0.5069, -0.0487],
        [ 0.3137, -0.1608,  0.2010,  0.7114, -0.0136, -1.0614, -0.3065,  0.0367,
         -0.2455,  0.2886,  0.4840,  0.9368, -0.6769,  0.2800,  0.1564, -0.4040,
         -0.4028, -0.7503,  0.2990, -0.2819, -0.9769,  0.6266,  0.0493,  0.1531,
         -0.0408,  0.0636, -0.5215, -0.2778,  0.5189,  0.2338,  0.1777],
        [ 0.5766,  0.0121,  0.6500,  0.2346, -0.0564, -0.2214,  0.4508,  0.6085,
          0.2080,  0.4678,  0.2434, -0.3284, -0.1483, -0.3799, -0.4861, -0.2664,
          0.0030, -0.3168, -0.2338,  0.2431,  0.0899, -0.4979, -0.6430,  0.4714,
          0.0061,  0.0960,  0.6067, -0.1599, -0.1262,  0.1542,  0.3333],
        [ 0.1029,  0.0600,  0.4567,  0.7435, -0.2666

In [35]:
path_contexts.index()

AttributeError: 'Tensor' object has no attribute 'index'

In [1]:
from FileReader import FileReader
import os

In [14]:

class FileReader:
    def __init__(self, filename):
        if not os.path.exists(filename) and os.path.isfile(filename):
            raise Exception("Not a valid filename")

        self.file = open(filename, "r")
        self.line_offsets = {}

        offset = 0
        line = self.file.readline()
        line_number = 0

        while line:
            self.line_offsets[line_number] = offset
            offset = offset + len(line)
            line = self.file.readline()
            line_number += 1

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            idx = list(range(idx.start or 0, idx.stop or len(self), idx.step or 1))
            return [self[i] for i in idx]
        else:
            self.file.seek(0)
            self.file.seek(self.line_offsets[idx])

            return self.file.readline().rstrip()

    def __len__(self):
        return len(self.line_offsets)

    def __del__(self):
        self.file.close()

data_root = "./data/java-small/"
train_file_path = os.path.join(data_root, "java-small.train.c2s")
test_file_path = os.path.join(data_root, "java-small.test.c2s")
dict_file_path = os.path.join(data_root, "java-small.dict.c2s")
        

f = FileReader(test_file_path)

In [15]:
f[:2]

['get|timestamp override,Nm0|MarkerExpr|Mth|Prim1,long override,Nm0|MarkerExpr|Mth|Nm2,METHOD_NAME long,Prim1|Mth|Nm2,METHOD_NAME long,Prim1|Mth|Bk|Ret|Nm0,timestamp METHOD_NAME,Nm2|Mth|Bk|Ret|Nm0,timestamp',
 'get|type override,Nm0|MarkerExpr|Mth|Cls1,type override,Nm0|MarkerExpr|Mth|Nm2,METHOD_NAME type,Cls1|Mth|Nm2,METHOD_NAME type,Cls1|Mth|Bk|Ret|Nm0,type METHOD_NAME,Nm2|Mth|Bk|Ret|Nm0,type']

In [16]:
len(f)

57088

In [20]:
x = torch.rand(4* 3, 2).flatten()

In [27]:
mask = (x == 0.3357)

In [40]:
def sequence_mask(lengths, maxlen=None, dtype=torch.bool):
    if maxlen is None:
        maxlen = lengths.max()
    row_vector = torch.arange(0, maxlen, 1)
    matrix = torch.unsqueeze(lengths, dim=-1)
    mask = row_vector < matrix

    mask.type(dtype)
    return mask

sample = randPaddedContexts(4, 3, 5)

In [41]:
sample = torch.tensor([
    [4, 5, 6, 0],
    [2, 3, 0 ,0]
])

lengths = [3, 2]

In [44]:
masks = sequence_mask(torch.tensor(lengths), maxlen=4)

In [45]:
sample[masks]

tensor([4, 5, 6, 2, 3])

In [4]:
a = torch.rand(3, 4, 5)
b = torch.rand(3, 4, 5)

In [5]:
a

tensor([[[0.2331, 0.8801, 0.5692, 0.5974, 0.3704],
         [0.3491, 0.0106, 0.4078, 0.8475, 0.1214],
         [0.8971, 0.6593, 0.7773, 0.2005, 0.3652],
         [0.4523, 0.2859, 0.1618, 0.0165, 0.1110]],

        [[0.6816, 0.0917, 0.1544, 0.4621, 0.0504],
         [0.9050, 0.5085, 0.0426, 0.5298, 0.7987],
         [0.9161, 0.1003, 0.8156, 0.9963, 0.2665],
         [0.2426, 0.1371, 0.8620, 0.8919, 0.2236]],

        [[0.6881, 0.7284, 0.4124, 0.9517, 0.3423],
         [0.4180, 0.1295, 0.4357, 0.0583, 0.8912],
         [0.0814, 0.3708, 0.3269, 0.3841, 0.6103],
         [0.7462, 0.5383, 0.8795, 0.3500, 0.2957]]])

In [6]:
b

tensor([[[0.2361, 0.0671, 0.8564, 0.2366, 0.7370],
         [0.4328, 0.3619, 0.6318, 0.0997, 0.8898],
         [0.2398, 0.0861, 0.4448, 0.2729, 0.6475],
         [0.4641, 0.4559, 0.9215, 0.8726, 0.8068]],

        [[0.5168, 0.6759, 0.1737, 0.8454, 0.1579],
         [0.1556, 0.8148, 0.3759, 0.7560, 0.6981],
         [0.0793, 0.1141, 0.5935, 0.8008, 0.9047],
         [0.0982, 0.9480, 0.2922, 0.6172, 0.5895]],

        [[0.1245, 0.9845, 0.8202, 0.4375, 0.3517],
         [0.6666, 0.7005, 0.7619, 0.7581, 0.4217],
         [0.7488, 0.6469, 0.4195, 0.6232, 0.7563],
         [0.3748, 0.6338, 0.8799, 0.7805, 0.8928]]])

In [10]:
torch.cat((a, b), dim=2).shape

torch.Size([3, 4, 10])

In [12]:
a.sum(dim=1).shape

torch.Size([3, 5])

In [46]:
import pandas as pd
import numpy as np

In [47]:
def load_counts(filename):
    freqs = {}
    i = 0
    with open(filename, "r") as f:
        for line in f:
            token, count = line.split("|")
            freqs[token] = int(count)
            i += 1
    return freqs
    
# os.listdir("./data/java-test")[:4]

In [48]:
target_count = load_counts("./data/java-test/target_count.txt")
node_count = load_counts("./data/java-test/node_count.txt")
subtoken_count = load_counts("./data/java-test/subtoken_count.txt")

In [53]:
def trim_by_quantile(kv, threshold=0.25):
    q = np.quantile(np.array(list(kv.values())), 0.25)
    return {k:v for k,v in kv.items() if v > q}

In [54]:
target_count = trim_by_quantile(target_count)
node_count = trim_by_quantile(node_count)
subtoken_count = trim_by_quantile(subtoken_count)

UnboundLocalError: local variable 'q' referenced before assignment

In [16]:
len(node_count)

344

In [18]:
import pickle
with open("./data/java-test/java-test.dict.c2s", 'wb') as file:
    pickle.dump(subtoken_count, file)
    pickle.dump(node_count, file)
    pickle.dump(target_count, file)
    pickle.dump(200, file)
    pickle.dump(57000, file)

In [34]:
sorted_subtoken = [{"token": k, "freq": v} for k, v in sorted(subtoken_count.items(), key=lambda item: item[1])]

In [36]:
df = pd.DataFrame(sorted_subtoken)

In [43]:
df.freq.quantile(0.5)

26.0

In [44]:
df[df.freq > 13].shape[0]

111656

In [45]:
df.shape[0]

153022